In [501]:
import pandas as pd
import sys

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
%matplotlib inline
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv1D, UpSampling1D
from keras.layers.pooling import MaxPooling1D
from keras.utils import np_utils

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split


In [413]:
#args = sys.argv
#PATH = args[1]

# jupyter notebookのときはここで指定
PATH = '../Data_block/analysis_by_programs/20170130ar/12ch/MAL5/'



In [551]:
# file_rest = 'TDAvec_autocor_rest.csv'
# file_tap = 'TDAvec_autocor_tap.csv'

file_rest = 'raw_rest.csv'
file_tap = 'raw_tap.csv'

# 読み込みたいファイルのパス
PATH_rest = PATH + file_rest
PATH_tap = PATH + file_tap

# csvファイル読み込み
rest = pd.read_csv(PATH_rest, header = 0)
tap = pd.read_csv(PATH_tap, header = 0) 

rest = rest.T
tap = tap.T

# RestとTappingのデータをまとめる
all_data = pd.concat([rest, tap], axis = 0)

# ベクトル化
X = all_data.as_matrix()

# ラベル作成
label_rest = np.zeros(len(rest.index))
label_tap = np.ones(len(tap.index))

y = np.r_[label_rest, label_tap]

k = 3

In [562]:
    
    # 識別率を格納する配列
    kCVscore = np.zeros(k)
    
    # ベクトルの長さを格納しておく
    bach_size = X.shape[1]
    
    # 分割数
    kf = KFold(n_splits=k, shuffle=False)
    # 繰り返し回数
    i = 0

    for train_index, eval_index in kf.split(X):
        
        print(i)
        
        X_train, X_test = X[train_index], X[eval_index]
        y_train, y_test = y[train_index], y[eval_index]

        # （データ数, ベクトルの長さ，1）という形にリシェイプする
        X_train = np.reshape(X_train, (-1, bach_size, 1))
        X_test = np.reshape(X_test, (-1, bach_size, 1))

        # ダミー変数に変換
        y_train = np_utils.to_categorical(y_train, 2)
        y_test = np_utils.to_categorical(y_test ,2)

        # 1次元CNNのインスタンスを作成
        # 参考文献（Time series classification via TDA）に従ってパラメータを設定
        # 1st Conv1d : kernel_number = 7, kernel_size = 6
        # 1st MacPooling : kernel_number = 7
        # 2nd Conv1d : kernel_number = 7, kernel_size = 2
        # 2nd MacPooling : kernel_number = 3
        # Flattenで1次元に
        model = Sequential()
        model.add(Conv1D(7, 6, padding='same', input_shape=(bach_size, 1)))
        model.add(Activation('relu'))
        model.add(MaxPooling1D(7, padding='same'))
        model.add(Conv1D(7, 2, padding='same', activation='relu'))
        model.add(MaxPooling1D(3, padding='same'))

        model.add(Flatten())
        model.add(Dense(units=2, activation='softmax'))
        model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])

        # 学習
        history = model.fit(X_train, y_train, epochs=10)
        
        # 評価結果（識別率）を格納
        score = model.evaluate(X_test, y_test)
        kCVscore[i] = score[1]
        print(score[1])
        i = i + 1

0
Epoch 1/10
13/13 [==============================] - 1s 103ms/step - loss: 0.2369 - acc: 0.7692
Epoch 2/10
13/13 [==============================] - 0s 428us/step - loss: 0.2352 - acc: 0.7692
Epoch 3/10
13/13 [==============================] - 0s 424us/step - loss: 0.2334 - acc: 0.7692
Epoch 4/10
13/13 [==============================] - 0s 360us/step - loss: 0.2317 - acc: 0.8462
Epoch 5/10
13/13 [==============================] - 0s 440us/step - loss: 0.2300 - acc: 0.8462
Epoch 6/10
13/13 [==============================] - 0s 365us/step - loss: 0.2282 - acc: 0.9231
Epoch 7/10
13/13 [==============================] - 0s 542us/step - loss: 0.2265 - acc: 0.8462
Epoch 8/10
13/13 [==============================] - 0s 547us/step - loss: 0.2249 - acc: 0.8462
Epoch 9/10
13/13 [==============================] - 0s 385us/step - loss: 0.2233 - acc: 0.8462
Epoch 10/10
7/7 [==============================] - 1s 110ms/step
0.142857149243
1
Epoch 1/10
13/13 [==============================] - 1s 105ms/

In [563]:
kCVscore

array([ 0.14285715,  0.5714286 ,  1.        ])